In [ ]:
import mne
import fooof
import numpy as np

raw = mne.io.read_raw_fif('data/sub-01_ses-02_raw.fif')
raw.load_data()
mat_data = raw.get_data()

In [ ]:
def ApEx(data, sfreq, ch_names, ch_types, freq_aMethod = 'multitaper', fmin = 0, fmax = np.inf, visualCheck = False):
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    mneRaw = mne.io.RawArray(data,info)
    psd, freq = mneRaw.compute_psd(method=freq_aMethod, fmin=fmin, fmax=fmax).get_data(return_freqs=True)
    
    exponents = []
    for ipsd in psd:
        fm = fooof.FOOOF()
        fm.fit(freq, ipsd)
        exponents.append(fm.get_params('aperiodic_params', col='exponent'))
        if visualCheck:
            fm.report(freq, ipsd, plt_log = True)

    return np.array(exponents)

In [ ]:
# For conversion to .fif type
sfreq = 512
ch_names = [str(i) for i in range(len(mat_data))]
ch_types = 'eeg'

# For Spectral Analysis
freq_aMethod = 'multitaper'                         # 'welch' or 'multitaper' 
fmin = 1
fmax = 30

exponents = ApEx(mat_data, sfreq, ch_names, ch_types, freq_aMethod, fmin, fmax, visualCheck = False)